# Oasis FM Test Tool v0.6

This notebook allows example insurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included

The following terminology is defined in OED. Please visit https://github.com/Simplitium/OED for more information.

* Standard direct insurance modelling 
    * Site (Coverages: Buildings, Other, Contents, BI, PD and All)
        * Ded, Limit
    * Cond (Coverages: All) A single condition per site only.
        * MinDed, Limit
    * Pol (Coverages: All)
        * Ded, MinDed, MaxDed, Limit
    * Layers - Multiple per account
        * Attachment, Limit, Participation
    * Deductible Types 0 and 2 (% TIV) only
    * Limit Types 0 and 2 (% TIV) only
    

## Not included
* Peril codes
* Type 1 terms (% loss)
* Code field


## Test cases

A selection of test cases can be found in the examples folder.


## Input files 

Files must be named the following:

* account.csv
* location.csv


## Validation Rules

* TBC

In [ ]:
# 1. Notebook and Python setup
%config IPCompleter.greedy=True

# Standard Python libraries
import io
import json
import os
import shutil

# 3rd party Python libraries
import jupyter_helper
import pandas as pd
import six
    
# Oasis imports
from oasislmf.model_preparation import oed
from oasislmf.manager import OasisManager as manager

In [ ]:
jupyter_helper.file_uploader('examples/uploaded')

In [ ]:
# 2. Load the OED for a worked example, in this case FM Test Case 3.

# Note that only the currently used fields are shown unless show_all is set to True. 
oed_dir = os.path.abspath('ftest/fm3')

# Account file
oed_account_file = os.path.join(oed_dir, "account.csv")
if not os.path.exists(oed_account_file):
    print("Path does not exist: {}".format(oed_account_file))
    exit(1)
account_df = pd.read_csv(oed_account_file)

# Location file
oed_location_file = os.path.join(oed_dir, "location.csv")
if not os.path.exists(oed_location_file):
    print("Path does not exist: {}".format(oed_location_file))
    exit(1)
location_df = pd.read_csv(oed_location_file)

In [ ]:
# 3. View/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

In [ ]:
# 4. View/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

In [ ]:
# 5. Create a persistent run directory, and pick up any edits in the grid before running the analysis
# and copy the source files into the run directory
run_dir = os.path.join(os.getcwd(), 'run_direct_dir')
if not os.path.exists(run_dir):
    os.mkdir(run_dir)

account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()

account_df.to_csv(path_or_buf=os.path.join(run_dir, "account.csv"), encoding='utf-8', index=False)
location_df.to_csv(path_or_buf=os.path.join(run_dir, "location.csv"), encoding='utf-8', index=False)

In [ ]:
# 8. Generate the direct losses
loss_factor = 1.0
_, ils, _ = manager().run_deterministic(run_dir, output_dir=run_dir, loss_percentage_of_tiv=loss_factor)

In [ ]:
# 9. View the direct losses.
ils

## Optional Steps


In [ ]:
# 10. Write the direct losses to file (in `run_reinsurance`)
ils.to_csv(path_or_buf=os.path.join(run_dir, 'output_direct.csv'), encoding='utf-8', index=False)